In [ ]:
# Universal Setup for Backend Environment
import sys
import os
import subprocess
from pathlib import Path

def setup_environment():
    """Setup the environment by installing necessary dependencies and setting paths."""
    # Get the backend directory. If we are in 'backend', it is cwd.
    backend_dir = Path.cwd()
    if backend_dir.name != 'backend':
        # Search for backend
        if (backend_dir / 'backend').exists():
             backend_dir = backend_dir / 'backend'
        elif (backend_dir.parent / 'backend').exists():
             backend_dir = backend_dir.parent / 'backend'
    
    # Add src to path if it exists (for 'from agent import ...' style)
    src_dir = backend_dir / 'src'
    if src_dir.exists():
        if str(src_dir) not in sys.path:
            sys.path.append(str(src_dir))
            print(f"  [OK] Added {src_dir} to sys.path")
    
    if str(backend_dir) not in sys.path:
        sys.path.append(str(backend_dir))
        
    # Verify backend/agent can be imported
    try:
        import agent
        print("  [OK] Agent module found and imported.")
    except ImportError:
        print("  [!] Agent module not found. Installing dependencies...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-e", str(backend_dir)])
        print("  [OK] Backend installed in editable mode.")

setup_environment()

In [ ]:
# --- MODEL CONFIGURATION ---
# @title Select Gemini Model
# @markdown Choose the Gemini model to use. Only Gemini 2.5 models are currently accessible via the API.

MODEL_STRATEGY = "Gemini 2.5 Flash (Recommended)" # @param ["Gemini 2.5 Flash (Recommended)", "Gemini 2.5 Flash-Lite (Fastest)", "Gemini 2.5 Pro (Best Quality)"]

import os

# Map selection to model ID
# Note: Gemini 1.5 and 2.0 models are deprecated/not accessible via this API
if MODEL_STRATEGY == "Gemini 2.5 Flash (Recommended)":
    SELECTED_MODEL = "gemma-3-27b-it"
elif MODEL_STRATEGY == "Gemini 2.5 Flash-Lite (Fastest)":
    SELECTED_MODEL = "gemma-3-27b-it-lite"
elif MODEL_STRATEGY == "Gemini 2.5 Pro (Best Quality)":
    SELECTED_MODEL = "gemma-3-27b-it"
else:
    # Default fallback
    SELECTED_MODEL = "gemma-3-27b-it"

print(f"Selected Model: {SELECTED_MODEL}")
print(f"Strategy: {MODEL_STRATEGY}")

# Set Environment Variables to override defaults
os.environ["QUERY_GENERATOR_MODEL"] = SELECTED_MODEL
os.environ["REFLECTION_MODEL"] = SELECTED_MODEL
os.environ["ANSWER_MODEL"] = SELECTED_MODEL
os.environ["TOOLS_MODEL"] = SELECTED_MODEL

# Ensure GOOGLE_API_KEY is set if GEMINI_API_KEY is present (for LangChain compatibility)
if "GEMINI_API_KEY" in os.environ:
    os.environ["GOOGLE_API_KEY"] = os.environ["GEMINI_API_KEY"]
    print("  [OK] Synced GEMINI_API_KEY to GOOGLE_API_KEY for LangChain")

In [ ]:
# --- MODEL VERIFICATION (Optional) ---
# @title Verify Model Configuration
# @markdown Run this cell to verify that your API key is configured correctly and the selected model is available.

import os

# Check if API key is set
if "GEMINI_API_KEY" not in os.environ:
    print("⚠️  GEMINI_API_KEY not found in environment variables!")
    print("   Please set it before proceeding:")
    print("   export GEMINI_API_KEY='your-api-key-here'")
else:
    try:
        from google import genai
        
        # Initialize the client
        client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])
        
        # Test the selected model
        print(f"🧪 Testing model: {SELECTED_MODEL}")
        response = client.models.generate_content(
            model=SELECTED_MODEL,
            contents="Explain how AI works in a few words"
        )
        
        print(f"  [OK] Model verification successful!")
        print(f"   Model: {SELECTED_MODEL}")
        print(f"   Response: {response.text[:100]}...")
        
    except ImportError:
        print("  [!] google-genai package not installed!")
        print("   Installing now...")
        import subprocess
        import sys
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "google-genai"])
        print("  [OK] Installed! Please re-run this cell.")
        
    except Exception as e:
        print(f"  [X] Model verification failed: {e}")
        print(f"   This could mean:")
        print(f"   - Invalid API key")
        print(f"   - Model '{SELECTED_MODEL}' not available in your region")
        print(f"   - Quota/billing issues (for experimental models)")
        print(f"   - Network connectivity issues")

In [ ]:
# --- COLAB SETUP START ---
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    print("🔧 Running in Google Colab - Installing dependencies...")
    
    # 1. Clone the repository
    !rm -rf gemini-fullstack-langgraph-quickstart
    !git clone https://github.com/MasumRab/gemini-fullstack-langgraph-quickstart.git
    
    # 2. Navigate to the correct directory
    import os
    repo_name = "gemini-fullstack-langgraph-quickstart"
    target_dir = os.path.join(repo_name, "notebooks")
    
    if os.path.exists(target_dir):
        os.chdir(target_dir)
        print(f"  [OK] Changed directory to {os.getcwd()}")
    else:
        # Fallback to repo root if specific dir not found
        if os.path.exists(repo_name):
            os.chdir(repo_name)
            print(f"  [OK] Changed directory to {os.getcwd()} (Fallback)")
    
    # 3. Install Backend (Quietly)
    # We install from the backend directory which should be reachable
    # relative to current dir or absolute
    
    # Find backend relative to current position
    import sys
    if os.path.exists("backend"):
        !pip install -q -e backend
    elif os.path.exists("../backend"):
        !pip install -q -e ../backend
    elif os.path.exists("src"): # We might be IN backend
        !pip install -q -e .
        
    print("  [OK] Dependencies installed!")
else:
    print("  [OK] Running locally")
# --- COLAB SETUP END ---

# SOTA Framework Comparison

This notebook compares our Deep Research Agent architecture against state-of-the-art frameworks:
1. **FlowSearch**
2. **RhinoInsight**
3. **TTD-DR**

## 1. Feature Comparison

| Feature | Our Agent | FlowSearch | RhinoInsight | TTD-DR |
|---------|-----------|------------|--------------|--------|
| Architecture | Modular (Nodes) | Pipeline | Checklist | Tree-of-Thought |
| RAG Type | Hybrid (Vector+Graph) | Vector Only | Graph Only | Vector |
| Evidence Auditing | ✅ Yes | ❌ No | ✅ Yes | ❌ No |
| Subgoal Verification | ✅ Yes | ❌ No | ❌ No | ✅ Yes |
| MCP Integration | ✅ Yes | ❌ No | ❌ No | ❌ No |


## 2. Performance Analysis

Based on the DeepResearch-Bench metrics calculated in Notebook 3.

In [ ]:
import json
import os
import pandas as pd
import matplotlib.pyplot as plt

# Load our results
results_path = "../results/benchmark_run.json"
if os.path.exists(results_path):
    with open(results_path, 'r') as f:
        data = json.load(f)
        our_scores = data.get("final_scores", {})
else:
    # Dummy data for visualization if run not complete
    our_scores = {
        "pass_at_1_accuracy": 75.0,
        "evidence_quality": 82.0,
        "subgoal_completion": 88.0,
        "hallucination_rate": 5.0,
        "context_efficiency": 12.5
    }

# SOTA Baselines (Approximate from papers/leaderboards)
baselines = {
    "FlowSearch": {
        "pass_at_1_accuracy": 68.0,
        "evidence_quality": 75.0,
        "subgoal_completion": 70.0,
        "hallucination_rate": 12.0,
        "context_efficiency": 10.0
    },
    "RhinoInsight": {
        "pass_at_1_accuracy": 72.0,
        "evidence_quality": 85.0,
        "subgoal_completion": 65.0,
        "hallucination_rate": 8.0,
        "context_efficiency": 11.0
    }
}

# Prepare DataFrame
metrics = ["pass_at_1_accuracy", "evidence_quality", "subgoal_completion"]
df_data = {"Metric": metrics}

# Add Our Scores
df_data["Our Agent"] = [our_scores.get(m, 0) for m in metrics]

# Add Baselines
for name, scores in baselines.items():
    df_data[name] = [scores.get(m, 0) for m in metrics]

df = pd.DataFrame(df_data)
df = df.set_index("Metric")

print("Performance Comparison Table:")
print(df)

# Visualization
try:
    df.plot(kind="bar", figsize=(10, 6))
    plt.title("Deep Research Agent vs SOTA")
    plt.ylabel("Score (%)")
    plt.ylim(0, 100)
    plt.grid(axis='y')
    plt.xticks(rotation=0)
    plt.show()
except ImportError:
    print("Matplotlib not available for plotting.")

## 3. Pending Comparisons (TODO)

### TODO: Update Feature Comparison Table
- [ ] Add row for 'Recursive Research' once `research_subgraph` is live.
- [ ] Add row for 'Structured Reading' once `content_reader` is live.

### TODO: Live Data Integration
- [ ] Replace dummy scores in cell above with real data loaded from `../results/benchmark_run.json` after a full `DeepResearch-Bench` run.